# Huấn luyện trên bộ dữ liệu ViCTSD
* Là dữ liệu thu thập được từ các bài báo
* Bình luận không mang nặng xu hướng công kích cá nhân, mà thiên vì phát biểu cảm nghĩ, đưa ra ý kiến

# Import library

In [172]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import pyvi

# Setup data

## Data path

In [173]:
TRAIN_PATH = '../Dataset_Cleaned/clean_train_victsd.csv'
DEV_PATH = '../Dataset_Cleaned/dev_victsd.csv'
TEST_PATH = '../Dataset_Cleaned/test_victsd.csv'

In [174]:
train = pd.read_csv(TRAIN_PATH)
dev = pd.read_csv(DEV_PATH)
test = pd.read_csv(TEST_PATH)

## Preprocess data

* Tokenize
* Remove stopwords
* pad_sequences

## Setup functions to preprocess

In [175]:
# word seqmentation
# ML không bắt buộc seqmentation
from pyvi import ViTokenizer, ViPosTagger
def tokenize(text):
    """
    Thật tuyệt vời -> Thật tuyệt_vời
    """
    return ViTokenizer.tokenize(text)

In [176]:
import os
with open('..\Stopwords\\vietnamese-stopwords-dash.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]

# Loại các từ stopwords không hợp lí - ảnh hưởng nhiều tới ý nghĩa câu
sus_stopwords = ["không","không_có","không_thể","chưa"]
for sus_stopword in sus_stopwords:
  stopwords.remove(sus_stopword)


def lower_and_remove_stopwords(text: str):
    """
    Loại bỏ các từ stopwords và chuyển văn bản về chữ thường
    Đây là con_chó -> con_chó"""
    # Chuyển đổi văn bản sang chữ thường
    text = text.lower()

    # Tách văn bản thành danh sách các từ
    words = text.split()

    # Loại bỏ stopword
    filtered_words = [word for word in words if word not in stopwords]

    # Ghép danh sách các từ đã lọc lại thành văn bản
    filtered_text = ' '.join(filtered_words)
    return filtered_text


In [177]:
import re

def remove_links_hashtag_tag(text):
    """
    Removes URLs from a text string.

    Args:
        text: The text string to process.

    Returns:
        The text string with URLs removed.
    """
    link_remover = r"(https?://[^\s]+)"
    hashtag_remover = r"# [^\s]+"
    tag_remover = r"@ [^\s]+"

    text = re.sub(link_remover, "", text)
    text = re.sub(hashtag_remover, "", text)
    text = re.sub(tag_remover, "", text)
  
    return text

### Test dữ liệu sau khi process dạng text

In [178]:
test_text = "bánh này không ngon"

In [179]:
test_text = ViTokenizer.tokenize(test_text)
test_text

'bánh này không ngon'

In [180]:
print(f"Before: {test_text}")
test_text = lower_and_remove_stopwords(test_text)
test_text = remove_links_hashtag_tag(test_text)
print(f"After: {test_text}")

Before: bánh này không ngon
After: bánh không ngon


### Get text and label each DataFrame

In [181]:
X_train = train['text'].astype(str)
y_train = train['label']
X_dev = dev['text'].astype(str)
y_dev = dev['label']
X_test = test['text'].astype(str)
y_test = test['label']

In [182]:
X_train[:5]

0                                       thật tuyệt vời
1    mỹ đã tuột dốc quá nhiều rồi giờ muốn vực dậy ...
2    tôi thấy người lái xe hơi bấm còi mới là người...
3    coi dịch là giặc đã mang tên đó mà xâm nhập vn...
4    thương các bé quá các con còn quá nhỏ mà đã ph...
Name: text, dtype: object

### Merge DataFrame

## Create train, test

X_train, X_test, y_train, y_test

In [183]:
def preprocess_data(path):
    """
    1. Get data from paths
    3. Tokenize -> lower and remove stopwords -> remove links, hashtags, tags
    4. Get X, y from dataframe (input, output)
    """
    train_data = pd.read_csv(path)[['text', 'label']]
    df_joined = pd.concat([train_data], ignore_index=True)
    df_joined['text'] = df_joined['text'].astype(str)
    df_joined['text'] = df_joined['text'].apply(tokenize)
    df_joined['text'] = df_joined['text'].apply(lower_and_remove_stopwords)
    df_joined['text'] = df_joined['text'].apply(remove_links_hashtag_tag)
    X = df_joined['text']
    y = df_joined['label'].tolist()
    return X, y, df_joined


In [184]:
X_train, y_train, train_data = preprocess_data(TRAIN_PATH)
X_dev, y_dev, dev_data = preprocess_data(DEV_PATH)
X_test_, y_test_, test_data = preprocess_data(TEST_PATH)

In [185]:
X_train[:5]

0                                            tuyệt_vời
1                                  mỹ tuột_dốc vực dậy
2    lái xe_hơi bấm còi lịch_sự văn minhđường_xá ch...
3    coi dịch giặc xâm_nhập vn đầu_hàng cút xéo vn ...
4    thương bé rời cha_mẹ chia buồn gia_đình cầu_mo...
Name: text, dtype: object

In [186]:
# Để lấy dữ liệu cho nhanh chứ ko up lên git
train_data.to_csv('preprocessed_train_data.csv', index=False)
dev_data.to_csv('preprocessed_dev_data.csv', index=False)
test_data.to_csv('preprocessed_test_data.csv', index=False)


### Load data from csv

In [187]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size=50000
embedding_dim=64
max_length=140

train_data = pd.read_csv('preprocessed_train_data.csv')
dev_data = pd.read_csv('preprocessed_dev_data.csv')
test_data = pd.read_csv('preprocessed_test_data.csv')

X_train, y_train = train_data['text'].astype(str), train_data['label']
X_val, y_val = dev_data['text'].astype(str), dev_data['label']

# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length, padding='post',truncating='post')

# Thực hiện thay đổi test để đưa vào tính toán val_acc
X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val, maxlen=max_length, padding='post',truncating='post')

In [274]:
def accuracy_fn(y_true, y_pred):
  y_pred_rounded = torch.round(y_pred)  
  correct = torch.eq(y_true, y_pred_rounded).sum().item()
  acc = (correct/len(y_pred))*100
  return acc

from sklearn.metrics import accuracy_score, f1_score, classification_report
def f1_score_fn(y_true, y_pred):
  y_true = y_true.int()
  y_pred = torch.round(y_pred)
  y_pred = y_pred.int()
  y_true=y_true.tolist()
  y_pred = y_pred.tolist()
  f1_score_value = f1_score(y_true=y_true, y_pred=y_pred, pos_label=1, zero_division=0)
  # print(classification_report(y_true=y_true, y_pred=y_pred,zero_division=1))  
  return f1_score_value


In [275]:
from sklearn.metrics import f1_score
tmp1 = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
tmp2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
f1_score(tmp1, tmp2, average='weighted')

0.907258064516129

## DataLoader

In [276]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, x_encoded: np.ndarray, y_encoded: pd.core.series.Series):
        # Setup
        self.x_encoded = x_encoded
        self.y_encoded = y_encoded.tolist()
    
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.x_encoded[idx]), self.y_encoded[idx])
        # return (self.x_encoded[idx], self.y_encoded[idx])
        # return {'text': self.x[idx], 'label': self.y_encoded[idx]}
    
    def __len__(self):
        return self.x_encoded.shape[0]


In [277]:
train_data = CustomDataset(X_train, y_train)
test_data = CustomDataset(X_val, y_val)

In [278]:
len(train_data)

6970

In [279]:
from torch.utils.data import DataLoader
BATCH_SIZE=32
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)                            

In [280]:
train_dataloader

In [337]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocal_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocal_size, embedding_dim=embedding_dim )
        self.lstm = nn.LSTM(embedding_dim , hidden_dim, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.activate1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

        self.se = nn.Sequential(
            nn.Embedding(num_embeddings=vocal_size, embedding_dim=embedding_dim),
            nn.LSTM(embedding_dim , hidden_dim, batch_first=True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            # nn.ReLU(),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc1(x)
        x = self.activate1(x)
        x = self.fc2(x)
        x = self.softmax(x)
        x = torch.mean(x, dim=1)
        # print(x)
        return x


In [338]:
LSTM_Model = LSTMModel(vocal_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=16, output_dim=1)
LSTM_Model

LSTMModel(
  (embedding): Embedding(50000, 64)
  (lstm): LSTM(64, 16, batch_first=True)
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (activate1): ReLU()
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (activate2): ReLU()
  (softmax): Softmax(dim=1)
  (se): Sequential(
    (0): Embedding(50000, 64)
    (1): LSTM(64, 16, batch_first=True)
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Softmax(dim=None)
  )
)

In [339]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(LSTM_Model.parameters(), lr=0.001)

In [340]:
y_pred_record = []

In [341]:
# Write a training and evaluationg loop for model_1
torch.manual_seed(42)

# import tqdm for progress bar
from tqdm.auto import tqdm

# Train for longer
epochs = 10

# # Put data on the target device
# X_padded_sequences, y_train = torch.tensor(X_padded_sequences).to(device), torch.tensor(y_train).to(device)
# padded_val_sequences, y_test=  torch.tensor(padded_val_sequences).to(device), torch.tensor(y_test).to(device)

# Create training and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n------")
  ### Training
  train_loss=0
  LSTM_Model.train()
  # Add a loop to loop through the training batches
  for batch, (X, y) in enumerate(train_dataloader):
    # print(f"Batch: {batch}")

    # 1. Forward
    X = X.long()
    y_pred = LSTM_Model(X)

    # 2. Calculate the loss
    y_pred_record.append(y_pred)
    loss = loss_fn(y_pred.squeeze(), y.float().squeeze())
    train_loss += loss

    # 3.
    optimizer.zero_grad()

    # 4.
    loss.backward()

    # 5.
    optimizer.step()

    # Print out what's happening
    # if batch %50==0:
    #   print(f"Looked at {batch*len(X)}/{len(train_dataloader.dataset)} samples.")

  # Divide total train loss by length of train dataloader
  train_loss /= len(train_dataloader)

  ### Testing
  test_loss, test_acc, avg_f1_score = 0,0,0
  LSTM_Model.eval()
  with torch.inference_mode():
    for X_test_, y_test_ in test_dataloader:
      # 1. Forward pass
      X_test_ = X_test_.long()
      test_pred = LSTM_Model(X_test_)

      # 2. Calculate the loss (accumulatively)
      test_loss += loss_fn(test_pred.squeeze(dim=1), y_test_.float())
      # print(test_pred.shape)
      # 3. Calculate accuracy
      test_acc += accuracy_fn(y_true= y_test_.float(),
                              y_pred = test_pred.squeeze(dim=1))
      # 4. Calculate f1 score
      avg_f1_score += f1_score_fn(y_true= y_test_.float(),
                              y_pred = test_pred.squeeze(dim=1))
    # Calculate the test loss average per batch
    test_loss /= len(test_dataloader)
    avg_f1_score /= len(test_dataloader)
    # Calculate the test acc average per batch
    test_acc /= len(test_dataloader)

  # print out what happen
  print(f"\nTrain loss: {train_loss:.4f} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}, F1 score: {avg_f1_score:.4f}")
  # print(f"\nTrain loss; {train_loss:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
------

Train loss: 0.5444 | Test loss: 0.5775, Test acc: 88.4425, F1 score: 0.0000
Epoch: 1
------

Train loss: 0.5444 | Test loss: 0.5824, Test acc: 88.3433, F1 score: 0.0000
Epoch: 2
------

Train loss: 0.5445 | Test loss: 0.5799, Test acc: 88.3929, F1 score: 0.0000
Epoch: 3
------

Train loss: 0.5444 | Test loss: 0.5799, Test acc: 88.3929, F1 score: 0.0000
Epoch: 4
------

Train loss: 0.5440 | Test loss: 0.5799, Test acc: 88.3929, F1 score: 0.0000
Epoch: 5
------

Train loss: 0.5444 | Test loss: 0.5824, Test acc: 88.3433, F1 score: 0.0000
Epoch: 6
------

Train loss: 0.5450 | Test loss: 0.5775, Test acc: 88.4425, F1 score: 0.0000
Epoch: 7
------

Train loss: 0.5442 | Test loss: 0.5799, Test acc: 88.3929, F1 score: 0.0000
Epoch: 8
------

Train loss: 0.5442 | Test loss: 0.5799, Test acc: 88.3929, F1 score: 0.0000
Epoch: 9
------

Train loss: 0.5447 | Test loss: 0.5799, Test acc: 88.3929, F1 score: 0.0000


## Lưu model

In [342]:
torch.save(LSTM_Model.state_dict(), 'LSTM.pth')

## Load model

In [343]:
predict_model = LSTMModel(vocal_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=16, output_dim=1)
predict_model.load_state_dict(torch.load('LSTM.pth'))

<All keys matched successfully>

## Dự đoán

### Nhập câu muốn dự đoán

In [344]:
def predictV1(test_sentences):
    """
    Dùng model lưu để đánh giá test
    """
    test_seq = tokenize(test_sentences)
    test_seq = lower_and_remove_stopwords(test_seq)
    test_seq = remove_links_hashtag_tag(test_seq)
    test_seq = tokenizer.texts_to_sequences([test_seq])
    # print(test_seq)
    test_seq = pad_sequences(test_seq, maxlen=max_length, padding='post',truncating='post')
    # print(test_seq)
    return predict_model(torch.tensor(test_seq))

In [345]:
def predictV2(test_sentences):
    """
    Dùng model đang huấn luyện để đánh giá test
    """
    test_seq = tokenize(test_sentences)
    test_seq = lower_and_remove_stopwords(test_seq)
    test_seq = remove_links_hashtag_tag(test_seq)
    test_seq = tokenizer.texts_to_sequences([test_seq])
    # print(test_seq)
    test_seq = pad_sequences(test_seq, maxlen=max_length, padding='post',truncating='post')
    # print(test_seq)
    return LSTM_Model(torch.tensor(test_seq))

In [346]:
test_data

,text,label
0,không kẻ chẳng,1
1,đạp xe văn_minh . haizzzz,1
2,văn_hoá,0
3,đời ta mươi đời . mua xe phục_vụ đi_lại . linh...,0
4,"tước lái vĩnh_viễn đi . chạy lếu_láo , không c...",1
...,...,...
995,biệt_thự nhà_riêng to_đùng kia chủ ( ) đỗ xe k...,1
996,"ló khôn , tài cống_hiến tuyệt_vời xuất_sắc . b...",0
997,bánh trung_thu trứng muối . trẻ_con không bánh...,0
998,1 lớp 1 cải_cách . mục_tiêu gd tiểu_học làm_qu...,0


In [347]:
test_data = pd.read_csv('preprocessed_test_data.csv')

X_test, y_test = test_data['text'].astype(str), test_data['label']

In [348]:
X_test.shape

(1000,)

In [349]:
X_test[:5]

0                                       không kẻ chẳng
1                            đạp xe văn_minh . haizzzz
2                                              văn_hoá
3    đời ta mươi đời . mua xe phục_vụ đi_lại . linh...
4    tước lái vĩnh_viễn đi . chạy lếu_láo , không c...
Name: text, dtype: object

In [ ]:
num_record = len(X_test)
is_true = 0
true_0 = 0
false_0 = 0
true_1 = 0
false_1 = 0
test_predict = []
for i in range(len(X_test)):
    if predictV2(X_test[i])<0.5:
        test_predict.append(0)
        if y_test[i] == 0:
            true_0 += 1
            is_true += 1
        else:
            false_0 += 1
    else:
        test_predict.append(1)
        if y_test[i] == 1:
            true_1 += 1
            is_true += 1
        else:
            false_1 += 1
    

tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<MeanBackward1>)
tensor([[0.0071]], grad_fn=<Mean

In [ ]:
num_record, is_true, true_0, true_1,false_0 ,false_1

(1000, 890, 890, 0, 110, 0)

In [ ]:
print(classification_report(y_true=y_test, y_pred=test_predict))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94       890
           1       0.00      0.00      0.00       110

    accuracy                           0.89      1000
   macro avg       0.45      0.50      0.47      1000
weighted avg       0.79      0.89      0.84      1000



c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [ ]:
6708/num_record

6.708

In [ ]:
# "Đồ ngu, sao mày ngu thế?", 1
# "Con này xấu xí, béo ú, chẳng ra gì cả.", 1
# "Đồ đàn bà con gái chỉ biết ăn bám đàn ông.",
# "Sản phẩm này tệ hại, ai mua cũng hối hận.",
# "Mày hát dở tệ, đừng bao giờ hát nữa nhé.", 0 
# "Mày dám nói tao ngu à? Thử lặp lại lần nữa xem!",
# "Đời mày xuống lỗ rồi con ạ", 0
# "Ngu thì nín :))",
# "khôn như vợ chồng nhà này k biết bao giờ mới giỗ đầu bạn nhỉ :))", 0 
# "toàn cái bọn lúc vay muốn dc vc.khi xong vc rồi thì vẩy đuôi b oi.g
# "sản phẩm này bố ỉa vào mà mua"

In [ ]:
# import csv
# filename = "E:\\AI Project\\word2vec_vi_syllables_100dims.txt"
# chunksize = 10 ** 6
# for chunk in pd.read_csv(filename, chunksize=chunksize, on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf-8'):
#     # chunk is a DataFrame. To "process" the rows in the chunk:
#     for index, row in chunk.iterrows():
#         print(row)